<a href="https://colab.research.google.com/github/skevin-dev/NLP-Second-hackathon-AI-Chatbot/blob/main/webscraping_and_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re 


In [12]:
def DataExtraction(link):
  """ This function helps to extract links and title from the first of the parsed website 
  
  Args
  ----
  link:str
      link of the page to be scrapped 

  returns
  -------
  df:dataframe
     a dataset containing all information 
  """
  content =  requests.get(link).content
  soup =  BeautifulSoup(content, "html.parser")

  #get all the links in the website 
  links  = soup.find_all('a')

  # get appropriate links with their corresponding titles
  link_ = []
  title_ = []
  for i in links:
    try:
      if str(i['href'][0:5]) =='https':
        link_.append(i['href'])
        title_.append(i.text)

    except:
      pass
   
  
  # put the into pandas dataframe 
  df = pd.DataFrame()
  df['title'] = title_
  df['links'] = link

  return df

In [16]:
df = DataExtraction('https://www.rwandair.com/')

In [17]:

def preprocessing(titles : list):
  """
  preprocessing function that helps to process our data 

  """
  cleaned_titles = []
  for title in titles:
    title = title.lower()
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    title = url_pattern.sub(r'', title)
    title = html_pattern.sub(r'', title)
    title = re.sub(r"[^\w\d'\s]+", ' ', title)
    cleaned_titles.append(title)
  return cleaned_titles

df['title'] = preprocessing(df['title'])

In [27]:
def create_intents(df):
  question = """Guide me to {}?
  Where do you get {}?
  What is the link to the {}?
  How do you get {}?
  Who is in Charge of {}?"""

  intents = []

  for index in range(len(df)):
    intent = {}
    tag = df.iloc[index]['title']
    intent['tag'] = tag
    intent['patterns'] = list()
    intent['responses'] = [f'You can use this link {df.iloc[index].links} for more information']
    for i in range(len(question.split('\n'))):
      intent['patterns'].append(question.split('\n')[i].format(intent['tag']))

    intents.append(intent)
    
  file = {'intents': intents}

  return file

In [28]:
f = create_intents(df)

In [29]:
f

{'intents': [{'tag': 'flight status',
   'patterns': ['Guide me to flight status?',
    '  Where do you get flight status?',
    '  What is the link to the flight status?',
    '  How do you get flight status?',
    '  Who is in Charge of flight status?'],
   'responses': ['You can use this link https://www.rwandair.com/ for more information']},
  {'tag': 'track your cargo ',
   'patterns': ['Guide me to track your cargo ?',
    '  Where do you get track your cargo ?',
    '  What is the link to the track your cargo ?',
    '  How do you get track your cargo ?',
    '  Who is in Charge of track your cargo ?'],
   'responses': ['You can use this link https://www.rwandair.com/ for more information']},
  {'tag': ' wheel chair ',
   'patterns': ['Guide me to  wheel chair ?',
    '  Where do you get  wheel chair ?',
    '  What is the link to the  wheel chair ?',
    '  How do you get  wheel chair ?',
    '  Who is in Charge of  wheel chair ?'],
   'responses': ['You can use this link https